<a href="https://colab.research.google.com/github/kanincityy/comp_ling_exercises/blob/main/Copy_of_%5BPLIN0072%5D_Tutorial_4_A_feedforward_neural_network_for_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install otter-grader
!wget "https://sebschu.com/files/plin0072/tutorials/tutorial4/tests.zip" && unzip -o tests.zip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.3/142.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.2/154.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.0/115.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 10.8 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1

In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook()

# Tutorial 4: A feedforward neural network for classification

In this tutorial, you will implement your first neural network. We'll once again do this in the context of a part-of-speech tagger.

### Setup

Please first make a copy of this notebook into your personal Google Drive. You can do so by clicking on _File_ (top left corner) and choosing "Save a copy in Drive". Then, make sure that you run the two cells on top of the notebook before running any of the other cells. Otherwise you will not be able to run the automatic tests.


## A feedforward neural network for part-of-speech taggging

As we've discussed in Lectures 3 and 4, we can use feedforward neural networks for building multilabel classifiers (instead of the multinomial logistic regression classifiers that we've been using so far).

To do this, we'll need two major things (along with lots of little bits, many of which are already provided in this notebook).

1. We need to define a dataset class that builds a datastructure for the examples for us.

2. We need to define the structure of the neural network model.

This notebook will walk you through on how to do these things. In order to keep this manageable, I've already provided quite a bit of skeleton code to get you started. I'd recommend you still try to understand what I've done in the code that I already provided and I'd even suggest you try to implement your own network for a different task after you've completed this tutorial to get a sense of which moving parts are required.

### Your task: Loading word vectors

Run the following cell to download the GloVe embeddings to the folder `glove`.

In [ ]:
# Download the embeddings trained on 6B tokens from Wikipedia and news texts.
! mkdir -p glove && cd glove && wget https://nlp.stanford.edu/data/glove.6B.zip && unzip glove.6B.zip

--2025-02-08 16:14:08--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-02-08 16:14:08--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.05MB/s    in 2m 40s  

2025-02-08 16:16:49 (5.14 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflatin

We'll once again start by loding the word embeddings as we've done in previous weeks.  We'll use the 300-dimensional embeddings stored in `glove/glove.6B.300d.txt`.

However, this week, I'd like you to add 2 vectors to the embedding matrix after you've loaded it:

1. An unknown vector. This vector should be in the second-to-last row of the embedding matrix and its value should be the mean of all vectors in the embedding matrix. Add an entry with the key `"-UNK-"` to `word2index` with the correct row in the embedding matrix.

2. A zero vector for missing words. This vector should be in the last row of the embedding matrix and its value should be a vector with all zeros. Add an entry with the key `"-NONE-"` to `word2index` with the correct row in the embedding matrix.

The final embedding matrix should then be of shape `(400002, 300)`.

In [ ]:
import numpy as np

# TODO: load the embeddings
NO_OF_DIMENSIONS = 300
import numpy as np

# TODO load the embeddings
NO_OF_DIMENSIONS = 300
cols_to_use = list(range(1, NO_OF_DIMENSIONS + 1))
embeddings = np.loadtxt("glove/glove.6B.300d.txt", encoding="UTF-8", usecols=cols_to_use, comments=None)

word2index = {}

# TODO: populate the word2index dictionary
with open("glove/glove.6B.100d.txt", "r", encoding="UTF-8") as embedding_f:
  # this loops through the file and returns the line number in the file and the
  # contents of the line on each iteration
  for i, line in enumerate(embedding_f):
    cols = line.split(" ") # let's split the line into columns
    word = cols[0] # the first column contains the word
    word2index[word] = i # add the word to the mapping

# Convert embeddings list to a numpy array
embeddings = np.array(embeddings)

# Compute the -UNK- vector (mean of all vectors)
UNK_VECTOR = np.mean(embeddings, axis=0)

# Create the -NONE- vector (a zero vector)
NONE_VECTOR = np.zeros(NO_OF_DIMENSIONS, dtype=np.float32)

# Add -UNK- and -NONE- to word2index
word2index["-UNK-"] = len(embeddings)
word2index["-NONE-"] = len(embeddings) + 1

# Add -UNK- and -NONE- vectors to the embedding matrix
embeddings = np.vstack([embeddings, UNK_VECTOR, NONE_VECTOR])

# Verify the final embedding matrix shape
print(f"Final embedding matrix shape: {embeddings.shape}")
print(f"word2index has {len(word2index)} entries.")

Final embedding matrix shape: (400002, 300)
word2index has 400002 entries.


In [ ]:
grader.check("q1")

q1 results: All test cases passed!

### Implementing the Dataset

The first thing we'll have to do is implement a dataset class. This is a class that will massage the data we have into the right format so that we can input it to the neural network.

A dataset class is somewhat similar to a feature extractor, but we rarely do anything more sophisticated than figuring out what the indices of the words in an embedding matrix should be.

As usual, we'll first have to read in the data from the CoNLL-U files. Run the following two cells to define the `read_conllu_file` function and download the ConLL-U files, as we've been doing before.


In [ ]:
# A function to read in CoNLL-U files, which contain sentences annotated with
# their part-of-speech tags.
# See https://universaldependencies.org/format.html for more info on this file
# format.

def read_conllu_file(file_path):
  sentences = []
  unique_labels = set()

  with open(file_path, "r", encoding="UTF-8") as in_f:
    current_sentence = []
    for line in in_f:
      line = line.strip()
      # ignore lines starting with # (comments)
      if line.startswith("#"):
        continue

      # an empty line indicates the end of the sentence
      if line == "":
        sentences.append(current_sentence)
        current_sentence = []
        continue

      # split the line into its parts
      parts = line.split("\t")

      # extract the index (the first column)
      idx = parts[0]

      # check if this is a multi-word token or an empty node
      if "." in idx or "-" in idx:
        continue

      if len(parts) < 4:
        print(parts)
      # extract the word and the tag, i.e., the second and fourth column
      word, tag = parts[1], parts[3]

      unique_labels.add(tag)

      # append the word, tag pair to the current sentence
      current_sentence.append((word, tag))

  return sentences, unique_labels


In [ ]:
# Download annotated files. This downloads files for English but you can also change the paths to download files for other languages.
# See the repositories in https://github.com/UniversalDependencies for other languages.
! mkdir -p data && cd data && wget https://github.com/UniversalDependencies/UD_English-EWT/raw/refs/heads/master/en_ewt-ud-train.conllu
! cd data && wget https://github.com/UniversalDependencies/UD_English-EWT/raw/refs/heads/master/en_ewt-ud-dev.conllu


--2025-02-08 16:19:06--  https://github.com/UniversalDependencies/UD_English-EWT/raw/refs/heads/master/en_ewt-ud-train.conllu
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/refs/heads/master/en_ewt-ud-train.conllu [following]
--2025-02-08 16:19:06--  https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/refs/heads/master/en_ewt-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14190227 (14M) [text/plain]
Saving to: ‘en_ewt-ud-train.conllu’

en_ewt-ud-train.con 100%[===================>]  13.53M  --.-KB/s    in 0.05s   

202

### Your task: building a label2index dictionary

As a step towards building the Dataset class, we'll need a `label2index` dictionary which maps the part-of-speech tag to an index. The exact index does not matter, but assuming that there are $k$ unique part of speech tags in our training data, each part of speech tag should be assigned a unique number between 0 and $k-1$.

For example, if the only part-of-speech tags are "NOUN", "VERB", and "ADJ", then one possible `label2index` would be:

```python
  label2index = {
    "NOUN": 0,
    "VERB": 1,
    "ADJ": 2
  }
```

You can use th set `unique_labels` that contains all the unique POS tags in the training data to construct this index.

In [ ]:
train_data_sentences, unique_labels = read_conllu_file("data/en_ewt-ud-train.conllu")
label2index = {}

# TODO: Populate `label2index`
label2index = {label: idx for idx, label in enumerate(sorted(unique_labels))}

# print(label2index)

In [ ]:
grader.check("q2")

q2 results: All test cases passed!

### Your task: Complete the Dataset Class

The following cell already contains the skeleton of a class `POSDataset` that inherits from the `torch.utils.data.Dataset` class.

A dataset class needs to implement at least two methods:
* the `__len__(self)` method returns the number of examples in the dataset
* the `__getitem__(self, index)` method that returns the `index`-th example

In this skeleton, I've already implemented the constructor (the `__init__` method), a method `get_word_idx` that returns the index of a word in the embedding matrix, and the `__len__(self)` and the `__getitem__(self, index)` methods.

Your job is to implement the `extract_examples` method. This method should build two lists:

* `self.labels` which contains a list of the indices of the POS tag for each example (as stored in `label2index`)
* `self.examples` which contains a list of lists of the word indices within the context window. We'll consider a window size of 5 here, that is we include the two words right before the word that we want to tag, the word that we want to tag, and the two word after the one we want to tag.

For any word that does not exist (e.g., because the target word is the first word of the sentence and there are no previous words, or the target word is the second-to-last word of the sentence and there is no word 2 positions after the target word), use the index of the special `-NONE-` word that you've created above.



In [ ]:
import torch

from torch.utils.data import Dataset, DataLoader

class POSDataset(Dataset):
    """Loads POS tagging dataset to be used in a feed-forward neural network."""

    # The constructor which takes in the list of sentences
    # returned by read_connlu_file and the word2index and label2index
    # dictionaries.
    def __init__(self, sentences, word2index, label2index):
      # Initialise the class instances with dataset
        self.sentences = sentences
        self.word2index = word2index
        self.label2index = label2index
        self.extract_examples()

    # this is a method that returns the index of a word
    # using the word2index dictionary.
    # It returns the index of the "-UNK-" token
    # if there is no embedding for the word.
    #
    # You can call this method from other methods of this class
    # using self.get_word_idx(word)
    def get_word_idx(self, word):
      word = word.lower()
      if word in self.word2index:
        return self.word2index[word]
      elif word == "-NONE-":
        # Explicitly handle the "-NONE-" token
        return self.word2index["-NONE-"]
      else:
        # Default to "-UNK-"
        return self.word2index["-UNK-"]

    # this method should populate self.examples
    # and self.labels
    def extract_examples(self):
      window_size = 2
      self.examples = [] # list of lists each entry corresponds to 1 example
      self.labels = [] # index of label corresponding to example

      # Iterate through sentence in dataset
      for sentence in self.sentences:
        # Define n_words for each sentence length
        n_words = len(sentence)
        # Unpack words and labels from sentence
        words, labels = zip(*sentence)
        # Iterate through index of words
        for i in range(n_words):
          # Build context window, creates a cohesive example for each target word!
          context_window = []
          # TODO: populate self.examples and self labels
          # as described above
          # Iterate words within context window range
          for j in range(i-window_size, i+window_size+1):
            if j < 0 or j >= n_words:
              # If word out of bounds, use '-NONE-' token
              context_window.append(self.get_word_idx("-NONE-"))
            else:
              # If valid get index with get_word_idx function
              context_window.append(self.get_word_idx(words[j]))

          # Add context window to examples list
          self.examples.append(context_window)
          # Add label index = Look into how we access this!!!
          self.labels.append(self.label2index[labels[i]])

    # this method returns the number of examples
    def __len__(self):
        return len(self.examples)

    # this method returns one training example
    # Note that it converts everything into torch.Tensors.
    # These objects are ways to represent matrices in the pytorch internal format.
    def __getitem__(self, index):
        input_ids = torch.LongTensor([self.examples[index]])
        label_id = torch.LongTensor([self.labels[index]])
        return input_ids, label_id

train_dataset = POSDataset(train_data_sentences, word2index, label2index)

In [ ]:
grader.check("q3")

q3 results:
    q3 - 1 result:
        ❌ Test case failed
        Error at line 48 in test q3:
              assert x1.tolist() == [[400001, 400001, 318, 11, 34408]], test_dataclass_desc(4, x1.tolist(), "x1.tolist()")
        AssertionError: 

        The last line of the following test program failed.
        Make sure that your function returns exactly the same value
        as specified in the <b>assert</b> statement.

        assert len(train_dataset) == 204579
        x1, y1 = train_dataset[0]
        x2, y2 = train_dataset[1]
        <b>assert x1.tolist() == [[400001, 400001, 318, 11, 34408]]</b>

        x1.tolist() returned:
        [[400000, 400000, 318, 11, 34408]]

### Your task: Defining the model

Now finally, you'll have to define the model. You are expected to implement a neural network with one hidden layer here.

This involves two parts:

1. In the constructor (the `__init__` method) you have to define everything that has parameters, i.e.,  that has weight matrices.

2. In the forward function, you have to define how the final output value is computed.

I'd recommend looking at the code from [last week's lecture colab](https://colab.research.google.com/drive/1F95UO_3Vry8LExPce-wfltIeIYEWBINP#scrollTo=FbzW2JeADxXN) for how to do this.

In both functions, I've already defined the embedding layer. So you only have to process the data starting with the vector of 5 concanated word vectors.

Make sure to define the size of the layers based on the constructor variables `input_dimension`, `hidden_dimension`,  and `output_dimension`.

Also note that the loss function, the cross entropy loss, does not require the final step to use the softmax function. The `forward` function can therefore return the value of what you would pass through the `softmax` function instead of a list of probabilties.

Finally, we typically process a small number of examples at the same time, in so-called batches. Pytorch is quite good at automatically handling this but, for example, when we compute the input using the embedding matrix, we have to consider this.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# Look at lecture colab from Week 3
class FeedForwardTagger(nn.Module):
  # init defines everything that has a weight
  def __init__(self, input_dimension, hidden_dimension, output_dimension, embeddings):
    super().__init__()

    # This defines the embedding layer and intialises it with an existing embedding matrix
    self.embedding_layer = nn.Embedding.from_pretrained(torch.tensor(embeddings).to(dtype=torch.float32))

    # TODO: Define the remaining layers here
    self.hidden_layer = nn.Linear(input_dimension, hidden_dimension)
    self.output_layer = nn.Linear(hidden_dimension, output_dimension)

    # Dropout layers
    self.dropout1 = nn.Dropout(0.5)

  # the input x is a matrix of batch_size rows with 5 word indices in each row
  # defines computations with components from init function
  # Forward method defines the data flow through the network
  def forward(self, x):
    # The following two lines embed the tokens
    input = self.embedding_layer(x) # returns a tensor with shape batch_size x embedding_dimension
    # This turns embedding_dimension matrix into a single vector of embeddings
    input = input.flatten(start_dim=1) # shape: batch_size x embedding_dimension

    # TODO: compute h and the output here
    h = F.relu(self.hidden_layer(input)) # shape: batch_size x hidden_dimension
    output = self.output_layer(h) # shape: batch_size x output_dimension

    return output

In [ ]:
grader.check("q4")

q4 results: All test cases passed!

### Training the model

Now that we've defined everything, we are ready to train the model.

The code below defines the model, the loss function, the optimizer and then a `train` function that runs the training process for a set number of epochs.

Run the following two blocks to train your neural network. If you implemented everything correctly, you should see your loss go down with (almost) every iteration.

In [ ]:
from torch.utils.data import DataLoader

# The following few lines check whether a GPU is available, and if so,
# they run everything on a GPU which will be much faster.
device = "cpu"
if torch.cuda.is_available():
  device = "cuda"
print(f"Using {device} device")

# Let's define the batch_size, the window size, the output size, and the size of the hidden layer
# Experiment with different sizes for hidden layer, etc. to optimise model
# Consider adding second layer?
batch_size = 64
window_size = 5
output_size = len(label2index)
hidden_size = 200

# Let's define the model and move it to the GPU, if available.
model = FeedForwardTagger(window_size * NO_OF_DIMENSIONS, hidden_size, output_size, embeddings).to(device)
# We use a cross-entropy loss here
loss_fn = nn.CrossEntropyLoss().to(device)
# Finally, let's define the optimiser
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-5)

# This loads the data for training
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# The function for running model training
def train(model, loss_fn, optimizer, dataloader, epochs=150):
    # number of times we go through the training data
    # set the mode of the model to training so that parameters are being updated
    model.train()

    for i in range(epochs):

        # intialise the loss at the beginning of each epoch
        epoch_loss = 0

        # iterate through all training batches
        for X, y in dataloader:

            # move the input and the labels to the GPU, if we are using a GPU
            X = X.to(device)
            y = y.to(device).flatten()

            # reset the optimiser
            optimizer.zero_grad()
            # compute the current model predictions for the input X
            # this calls the forward function of the model
            y_hat = model(X)
            # compute the loss for the current predictions
            loss = loss_fn(y_hat, y)
            # perform backpropagation
            loss.backward()
            # and update the weights
            optimizer.step()

            # add the loss of the current batch to the loss of the epoch
            epoch_loss += loss.item()

        # print the loss at the end of every epoch
        print("Epoch: {0}, Loss: {1}, ".format(i, epoch_loss))







Using cuda device


Run the following cell to start the training process. You can also increase the number of epochs, which should improve your model but it will also take longer to run the training.

In [ ]:
train(model, loss_fn, optimizer, train_dataloader, epochs=15)

Epoch: 0, Loss: 1646.0208383053541, 
Epoch: 1, Loss: 866.9893937073648, 
Epoch: 2, Loss: 598.9228148944676, 
Epoch: 3, Loss: 440.39656713046134, 
Epoch: 4, Loss: 332.0264025107026, 
Epoch: 5, Loss: 249.4213908817619, 
Epoch: 6, Loss: 191.3998173205182, 
Epoch: 7, Loss: 146.97759441216476, 
Epoch: 8, Loss: 121.01833791239187, 
Epoch: 9, Loss: 98.57993226801045, 
Epoch: 10, Loss: 83.37917726230808, 
Epoch: 11, Loss: 73.1302848883206, 
Epoch: 12, Loss: 67.14821660402231, 
Epoch: 13, Loss: 60.23000479664188, 
Epoch: 14, Loss: 58.024878251279006, 


### Evaluating the model

Finally, let's evaluate how well the model does on the test data.

The following three cells construct the test dataset, then use the model go run predictions, and finally  

In [ ]:
# load the test sentences from the conllu file
test_data_sentences, _ = read_conllu_file("data/en_ewt-ud-dev.conllu")

# construct the test dataset from the test sentences
test_dataset = POSDataset(test_data_sentences, word2index, label2index)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)


In [ ]:
# this tells the model that it should only perform predictions and not update any weights
model.eval()

preds = []
true_labels = []
# loop through all batches in the test datseet
for X, y in test_dataloader:

  # move X to the correct device (the GPU if it exists)
  X = X.to(device)

  # make predictions by calling model(X) and move them back to the cpu, if we are using a GPU
  model_prediction = model(X).cpu()

  # this gets the index of the top 1 prediction, i.e., the class index of the most probable class
  y_hat = model_prediction.topk(1)[1]

  # add the true label and the predictions to the respective lists
  true_labels.extend(y.tolist())
  preds.extend(y_hat.tolist())


In [ ]:
# compute the accuracy

correct = 0
n = len(true_labels)
for i in range(n):
  if true_labels[i] == preds[i]:
    correct +=1

print(f"Test accuracy: {correct / n * 100:.2f}%")

Test accuracy: 91.77%


### Additional explorations

Once you've completed these steps, you can also experiment with optimising the model. For example, you can change the hyperparameters, these are parameters such as the dimension of the hidden layer or the window size and see how it affects model performance. You can also try training the model for more or fewer epochs and see how that affects performance. Or you can experiment with the model architecture, and for example, add additional hidden layers to the model.